In [668]:
import pandas as pd 
import json
import glob
import re 
CLEANR = re.compile('<.*?>') 


def check_source_row(row):
    head = row['head']
    sent = row['sent']
    if re.search('-\d', head):
        head = re.sub('-\d', '', head)
    
    heads = head.split(';')
    checks = []
    for x in heads:
        not_present = x.strip().lower() in ' '.join(sent).lower()
        checks.append(not_present)
        
    return all(checks)


def get_combined_df(annotated_fn, input_fn):
    json_dat = json.load(open(annotated_fn))['data']
    if isinstance(json_dat, dict) and 'row_data' in json_dat:
        json_dat = json_dat['row_data']
    annot_df = pd.DataFrame(json_dat)
    annot_df = annot_df.applymap(lambda x: x['field_value'] if isinstance(x, dict) else x)
    
    input_dat = json.load(open(input_fn))['html_data']
    input_df = pd.DataFrame(input_dat)

    annot_df_with_input = (
        input_df[['sent', 'sent_idx']]
             .merge(annot_df[['row_idx', 'head', 'quote_type', 'source_type']], left_on='sent_idx', right_on='row_idx')
             .drop(['row_idx', ], axis=1)
#      .loc[lambda df: df['sent'].str.strip().str.len() > 1]
    )
    
    return annot_df_with_input


def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

####### 
input_data_files = glob.glob('../app/data/input_data/*/*')
annotated_files = glob.glob('../app/data/output_data_affil-role/*/*')
checked_files = glob.glob('../app/data/checked_data_affil-role/*/*')
all_multiply_annotated_sentences = []
all_sources = []

for annot_fn in annotated_files:
    doc_id = re.search('\d+', annot_fn.split('/')[-1])[0]
    input_fn = annot_fn.replace('output_', 'input_').replace('_affil-role', '').replace('annotated-', 'to-annotate-')
    checked_cand = annot_fn.replace('output_', 'checked_').replace('annotated-', 'checked-')
    if checked_cand in checked_files:
        annot_fn = checked_cand
        
    annot_df_w_input = get_combined_df(annot_fn, input_fn)
    annot_df_w_input['doc_id'] = doc_id
    multiply_annotated = annot_df_w_input.loc[lambda df: df['head'].str.contains('-\d') == True]
    all_multiply_annotated_sentences.append(multiply_annotated)
    all_sources.append(annot_df_w_input)

all_sources_df = pd.concat(all_sources)
all_sources_df['sent'] = all_sources_df['sent'].apply(cleanhtml)


def cache_doc_tokens(input_doc, tokenizer, nlp):
    doc_tokens_by_word = []
    doc_tokens_by_sentence = []
    for sent, _, _, _ in input_doc:
        words = list(map(str, nlp(sent.strip())))
        enc = []
        for w_idx, w in enumerate(words):
            if w_idx == 0:
                add_prefix_space = False
            else:
                add_prefix_space = True
            enc.append(
                tokenizer.encode(w, add_special_tokens=False, add_prefix_space=add_prefix_space)
            )
        doc_tokens_by_word.append(enc)
        tokenized_sentence = [tokenizer.bos_token_id] + [i for l in enc for i in l] + [tokenizer.eos_token_id]
        doc_tokens_by_sentence.append(tokenized_sentence)
        
    doc_tokens = [i for l in doc_tokens_by_sentence for i in l]    
    word_lens_by_sent = [list(map(len, x)) for x in doc_tokens_by_word]
    word_lens_by_sent_cumsum = list(map(lambda x: np.cumsum([1] + x), word_lens_by_sent)) # we need a [1] offset
                                                                                          # in the cumsum because there 
                                                                                          # is an extra bos token added.
    sent_lens = list(map(len, doc_tokens_by_sentence))
    sent_lens_cumsum = np.cumsum([0] + sent_lens)
                         
    return (
        doc_tokens_by_word,
        doc_tokens_by_sentence,
        doc_tokens,
        word_lens_by_sent_cumsum,
        sent_lens,
        sent_lens_cumsum
    )

In [522]:
tokenizer.encode('hello', add_special_tokens=False, add_prefix_space=True)

[20760]

In [523]:
tokenizer.encode('there', add_special_tokens=False, add_prefix_space=True)

[89]

In [524]:
tokenizer.encode('hello', add_special_tokens=False, add_prefix_space=False)

[42891]

In [525]:
tokenizer.encode('there', add_special_tokens=False, add_prefix_space=False)

[8585]

In [528]:
tokenizer.decode([89])

' there'

In [527]:
tokenizer.decode([8585])

'there'

# Check which sources are not found in their directly-tagged sentences

In [286]:
unfound = (
    all_sources_df
        .loc[lambda df: df['head'] != '']
        .groupby(['doc_id', 'head'])
        .aggregate(list)
        .reset_index()
        .loc[lambda df: ~df.apply(check_source_row, axis=1)]
)

In [287]:
unfound['head'].value_counts().index.tolist()

['journalist',
 'passive-voice',
 'Joseph R. Biden Jr.',
 'Donald J. Trump',
 'Elizabeth Warren',
 'Trump',
 'David Cameron',
 'John J. Bates',
 'Mahmoud Badr; Mohammed Abdel-Aziz; Hassan Shahin; Mai Wahba; Mohammed Heikal',
 'Mai Wahba',
 'Ron DeSantis',
 'Trevor Haynes',
 'Fu Cheng Qiu',
 'Amy McGrath',
 'Sputnik news agency',
 'Mika Brzezinski',
 'I. Launa',
 'Donald Trump',
 ' Food and Drug Administration',
 'Vladimir V. Putin',
 'Austin Gilbert',
 'Jonathan Segal',
 'Mike Pence',
 'Ryan Ellis',
 'some men',
 'David W. Eaton; Xuewei Bao',
 "Kenya's top politicians",
 "mine's owners",
 'Mahmoud Badr',
 "Paul Ryan's spokesman",
 'Hassan Shahin',
 'bill-7',
 'Giulio Regeni',
 'epytians',
 'Gov. Kate Brown',
 'Oregon State Senate',
 "Oregon's house",
 'Vermont Senate Judiciary Committee',
 "bill's sponsors",
 'bill-2',
 'bill-3',
 'bill-4',
 'bill-8',
 'Cody Wilson ',
 'Jerome H. Powell',
 'Fire departement',
 'Asia Development Bank',
 'Mary Walsh',
 'Ohio State University',
 'campaign

# Tag sources in document

In [495]:
def find_rk(seq, subseq):
    n = len(seq)
    m = len(subseq)
    if seq[:m] == subseq:
        return 0
    hash_subseq = sum(hash(x) for x in subseq)  # compute hash
    curr_hash = sum(hash(x) for x in seq[:m])  # compute hash
    for i in range(1, n - m + 1):
        curr_hash += hash(seq[i + m - 1]) - hash(seq[i - 1])   # update hash
        if hash_subseq == curr_hash and seq[i:i + m] == subseq:
            return i
    return False

def get_source_in_sentence(source_head, sentence):
    if re.search('-\d', source_head):
        source_head = re.sub('-\d', '', source_head)
    if source_head in sentence:
        return find_rk(sentence.split(), source_head.split())
    else:
        return -1

In [515]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [656]:
def find_source_offset(source_head, source_sents, doc_sents, tok_lens_by_sent, sent_lens):
    # 1. iterate through source-related sentences first
    found = False
    for sentence, _, s_idx, _ in source_sents:
        sentence = unidecode(sentence)
        offset = get_source_in_sentence(source_head.lower(), sentence.lower())
        if offset != -1:
            sent_toks = tok_lens_by_sent[int(s_idx)]
            return {
                'source': source_head,
                's_idx': s_idx,
                'start_tok_idx': sent_lens[int(s_idx)] + sent_toks[offset],
                'end_tok_idx': sent_lens[int(s_idx)] + sent_toks[offset + len(source_head.split())],
                'doc_idx': doc_idx
            }

    # 2. iterate through the whole document if the source is not in the source sentences
    for sentence, _, s_idx, _ in doc_sents:
        sentence = unidecode(sentence)
        offset = get_source_in_sentence(source_head.lower(), sentence.lower())
        if offset != -1:
            sent_toks = tok_lens_by_sent[int(s_idx)]
            return {
                'source': source_head, 
                's_idx': s_idx,
                'start_tok_idx': sent_lens[int(s_idx)] + sent_toks[offset],
                'end_tok_idx': sent_lens[int(s_idx)] + sent_toks[offset + len(source_head.split())],
                'doc_idx': doc_idx
            }
        
    # 3. nothing found, returning
    return {
        'source': source_head, 
        's_idx': -1,
        'e_idx': -1,
        'start_tok_idx': -1,
        'end_tok_idx': -1,
        'doc_idx': doc_idx
    }

In [666]:
def generate_training_chunk_from_source_offset(source_offset_chunk, all_doc_tokens, sent_lens):
    s_idx = int(source_offset_chunk['s_idx'])
    
    ## 
    training_chunk = {}
    training_chunk['start_position'] = source_offset_chunk['start_tok_idx']
    training_chunk['end_position'] = source_offset_chunk['end_tok_idx']
    training_chunk['context'] = all_doc_tokens
    sent_inds = []
    for i, l in enumerate(sent_lens):
        if i == s_idx:
            sent_inds += [1] * l
        else:
            sent_inds += [0] * l 
    
    training_chunk['sentence_indicator_tokens'] = sent_inds
    return training_chunk

In [670]:
import numpy as np 
import csv, itertools
from tqdm.auto import tqdm
from unidecode import unidecode

data_path = '../models_neural/quote_attribution/data/our-annotated-data__stage-2.tsv'
split, data_chunk = [], []
with open(data_path) as f:
    csv_reader = csv.reader(f, delimiter="\t")
    csv_data = list(csv_reader)

grouped = []
for doc_idx, doc in itertools.groupby(csv_data, key=lambda x: x[3]):  # group by doc_id
    sorted_doc = sorted(doc, key=lambda x: int(x[2]))  # sort by sent_id
    sorted_doc = list(map(lambda x: [x[0].strip(), x[1], x[2], x[3]] , sorted_doc))
    grouped.append((doc_idx, sorted_doc))

### 
training_data = []
for doc_idx, doc_to_group in tqdm(grouped, total=len(grouped)):
    doc_to_group[0][0] = 'journalist passive-voice ' + doc_to_group[0][0]
    (
        doc_tok_by_word, 
        doc_tok_by_sent,
        all_doc_tokens, 
        word_len_cumsum,
        sent_lens,
        sent_len_cumsum
    ) = cache_doc_tokens(doc_to_group, tokenizer, nlp)    
    
    doc_to_group = sorted(doc_to_group, key=lambda x: x[1]) # sort by source
    
    for source_heads, source_sentences in itertools.groupby(doc_to_group, key=lambda x: x[1]):
        if source_heads == 'None':
            continue
        
        for source_head in source_heads.split(';'):
            source_head = unidecode(source_head).strip()
            source_chunk = find_source_offset(source_head, source_sentences, doc_to_group, word_len_cumsum, sent_len_cumsum)
            training_chunk = generate_training_chunk_from_source_offset(source_chunk, all_doc_tokens, sent_lens)
            training_data.append(training_chunk)

  0%|          | 0/296 [00:00<?, ?it/s]

In [658]:
source_to_word_offset[-1]

[{'source': 'Food and Drug Administration',
  's_idx': '11',
  'start_tok_idx': 419,
  'end_tok_idx': 423,
  'doc_idx': '/test/716'},
 {'source': 'Elizabeth Holmes',
  's_idx': '1',
  'start_tok_idx': 60,
  'end_tok_idx': 62,
  'doc_idx': '/test/716'},
 {'source': 'Theranos',
  's_idx': '0',
  'start_tok_idx': 4,
  'end_tok_idx': 5,
  'doc_idx': '/test/716'},
 {'source': 'Wall Street Journal',
  's_idx': '11',
  'start_tok_idx': 394,
  'end_tok_idx': 397,
  'doc_idx': '/test/716'},
 {'source': 'company spokesman',
  's_idx': '8',
  'start_tok_idx': 294,
  'end_tok_idx': 296,
  'doc_idx': '/test/716'}]

In [650]:
tokenizer.decode(doc_tok_by_sent[11][11:14])

' Wall Street Journal'

In [661]:
tokenizer.decode(all_doc_tokens[294:296])

' company spokesman'

In [685]:
import pandas as pd 
train_df = pd.read_csv('../models_neural/quote_attribution/data/our-annotated-data__stage-2.tsv', sep='\t', header=None)

In [687]:
train_df.loc[lambda df: df[1] == 'employee']

,0,1,2,3
13695,"Asked why it had taken so long for the structures to be demolished , an employee at Social , a nearby restaurant , who declined to give his name , scrunched up his face and laughed , saying it was bureaucracy .",employee,45,/train/658
15437,One involved a manager groping employees ’ breasts .,employee,36,/train/741


In [628]:
model_name = "deepset/roberta-base-squad2"

In [629]:
from transformers import AutoConfig

In [631]:
config= AutoConfig.from_pretrained(model_name)

In [632]:
config.num_labels

2

In [621]:
# do checks!!! clean up these sources!!! 

In [518]:
source_word_offset_df = pd.DataFrame([i for s in source_to_word_offset for i in s])

# this is likely because of rows with multiple sources (sep by ';')
source_word_offset_df.assign(c=1).groupby(['doc_idx', 'source'])['c'].sum().loc[lambda s: s>1].head()

## todo: go through and correct all these sources
_ = source_word_offset_df.loc[lambda df: df['s_idx'] == -1]

In [519]:
doc_to_group = sorted(doc_to_group, key=lambda x: int(x[2]))

In [566]:
doc_tok_by_word, doc_tok_by_sent, blank_toks_by_sent, all_doc_tokens, word_len_cumsum = cache_doc_tokens(
    doc_to_group, tokenizer, nlp
)

In [567]:
word_len_cumsum

[array([ 1,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8, 11, 12, 13, 14, 15, 16, 17, 18, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
        39, 40, 41, 42]),
 array([ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 14, 15, 16, 17, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]),
 array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 37, 38,
        39, 40, 41, 42]),
 array([ 1,  2,  3,  4,  6,  7,  8, 10, 11, 13, 14, 15, 18, 19, 20, 21, 23

In [568]:
tokenizer.decode(toks)

' Food and Drug Administration'

In [579]:
source_to_word_offset[-1][4]

{'source': 'company spokesman',
 's_idx': '8',
 'start_word_idx': 1,
 'end_word_idx': 3,
 'doc_idx': '/test/716'}

In [580]:
word_len_cumsum[8][1]

2

In [581]:
word_len_cumsum[8][3]

4

In [582]:
tokenizer.decode(doc_tok_by_sent[8][2:4])

' company spokesman'

# Make sure we have a good tokenizing pattern

In [132]:
from transformers import AutoTokenizer, RobertaTokenizer

In [653]:
tokenizer = RobertaTokenizer.from_pretrained('/Users/alex/.cache/torch/transformers/named-models/roberta-base-expanded-embeddings')

In [143]:
doc_sents = list(map(lambda x: x[0].strip(), sorted_doc))

In [144]:
doc_str = ' '.join(doc_sents)

In [148]:
s = doc_sents[0]

In [672]:
import torch

In [674]:
torch.ones_like(torch.tensor([1,2,3]))

tensor([1, 1, 1])

In [675]:
torch.ones_like([1,2,3])

TypeError: ones_like(): argument 'input' (position 1) must be Tensor, not list

In [677]:
import spacy
model = spacy.load("en_core_web_lg")
model._path

PosixPath('/Users/alex/opt/anaconda3/lib/python3.7/site-packages/en_core_web_lg/en_core_web_lg-3.3.0')

In [688]:
t_server = """{
  "_name_or_path": "/Users/alex/.cache/torch/transformers/named-models/roberta-base/",
  "accumulate_grad_batches": 1,
  "adam_beta1": 0.9,
  "adam_beta2": 0.999,
  "adam_epsilon": 1e-08,
  "add_cross_attention": false,
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "batch_size": 1,
  "bos_token_id": 0,
  "chunk_size_feed_forward": 0,
  "classifier_dropout": null,
  "concat_headline": false,
  "cross_attention_hidden_size": null,
  "decoder_start_token_id": null,
  "diversity_penalty": 0.0,
  "do_sample": false,
  "doc_embed_arithmetic": false,
  "downsample_negative_data": 1.0,
  "dropout": 0.1,
  "early_stopping": false,
  "embedding_dim": 768,
  "encoder_no_repeat_ngram_size": 0,
  "env": "local",
  "eos_token_id": 2,
  "eval_data_file": null,
  "experiment": "roberta_classification",
  "finetuning_task": null,
  "forced_bos_token_id": null,
  "forced_eos_token_id": null,
  "freeze_embedding_layer": false,
  "freeze_encoder_layers": [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9
  ],
  "freeze_pooling_layer": false,
  "freeze_transformer": false,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dim": 512,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "learning_rate": 0.0001,
  "length_penalty": 1.0,
  "local": false,
  "local_rank": -1,
  "log_all_metrics": false,
  "loss_weighting": null,
  "max_grad_norm": 0,
  "max_length": 20,
  "max_length_seq": 200,
  "max_num_sent_positions": 40,
  "max_num_sentences": 100,
  "max_num_word_positions": 2048,
  "max_position_embeddings": 2048,
  "min_length": 0,
  "model_type": "",
  "no_repeat_ngram_size": 0,
  "notes": "Stage 2: Quote Attribution. Classification. Our dataset only.",
  "num_attention_heads": 12,
  "num_beam_groups": 1,
  "num_beams": 1,
  "num_dataloader_cpus": 10,
  "num_documents": null,
  "num_gpus": 1,
  "num_hidden_layers": 12,
  "num_nodes": 1,
  "num_output_tags": 1,
  "num_return_sequences": 1,
  "num_token_types": 3,
  "num_train_epochs": 3,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_scores": false,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "prefix": null,
  "pretrained_lm_model_path": null,
  "pretrained_model_path": "./roberta-base-expanded-embeddings",
  "problem_type": null,
  "pruned_heads": {},
  "remove_invalid_values": false,
  "repetition_penalty": 1.0,
  "return_dict": true,
  "return_dict_in_generate": false,
  "sentence_contextualizer_model_type": "roberta",
  "sentence_embedding_method": "attention",
  "sep_token_id": null,
  "shuffle_data": false,
  "sinusoidal_embeddings": false,
  "spacy_model_file": "spacy/en_core_web_lg",
  "split_type": "random",
  "task_specific_params": null,
  "temperature": 1.0,
  "tie_encoder_decoder": false,
  "tie_word_embeddings": true,
  "tokenizer_class": null,
  "top_k": 50,
  "top_p": 1.0,
  "torch_dtype": "float32",
  "torchscript": false,
  "train_data_file": "data/our-annotated-data__stage-2.tsv",
  "train_on_none": false,
  "transformer_attention_probs_dropout_prob": 0.1,
  "transformer_hidden_dropout_prob": 0.1,
  "transformers_version": "4.12.2",
  "type_vocab_size": 1,
  "use_bfloat16": false,
  "use_cache": false,
  "use_cpu": false,
  "use_deepspeed": false,
  "use_doc_emb": false,
  "use_headline": false,
  "use_headline_embs": false,
  "use_positional": false,
  "vocab_size": 50265,
  "warmup_steps": 0,
  "weight_decay": 0
}

"""

In [689]:
t_local = """
{
  "_name_or_path": "/Users/alex/.cache/torch/transformers/named-models/roberta-base/",
  "accumulate_grad_batches": 1,
  "adam_beta1": 0.9,
  "adam_beta2": 0.999,
  "adam_epsilon": 1e-08,
  "add_cross_attention": false,
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "batch_size": 1,
  "bos_token_id": 0,
  "chunk_size_feed_forward": 0,
  "classifier_dropout": null,
  "concat_headline": false,
  "cross_attention_hidden_size": null,
  "decoder_start_token_id": null,
  "diversity_penalty": 0.0,
  "do_sample": false,
  "doc_embed_arithmetic": false,
  "downsample_negative_data": 0.1,
  "dropout": 0.1,
  "early_stopping": false,
  "embedding_dim": 768,
  "encoder_no_repeat_ngram_size": 0,
  "env": "local",
  "eos_token_id": 2,
  "eval_data_file": null,
  "experiment": "roberta_classification",
  "finetuning_task": null,
  "forced_bos_token_id": null,
  "forced_eos_token_id": null,
  "freeze_embedding_layer": false,
  "freeze_encoder_layers": [],
  "freeze_pooling_layer": false,
  "freeze_transformer": true,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dim": 512,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "learning_rate": 0.0001,
  "length_penalty": 1.0,
  "local": true,
  "local_rank": -1,
  "log_all_metrics": false,
  "loss_weighting": null,
  "max_grad_norm": 0,
  "max_length": 20,
  "max_length_seq": 200,
  "max_num_sent_positions": 40,
  "max_num_sentences": 100,
  "max_num_word_positions": 2048,
  "max_position_embeddings": 2048,
  "min_length": 0,
  "model_type": "",
  "no_repeat_ngram_size": 0,
  "notes": "sentence-classification",
  "num_attention_heads": 12,
  "num_beam_groups": 1,
  "num_beams": 1,
  "num_dataloader_cpus": 10,
  "num_documents": 30,
  "num_gpus": 0,
  "num_hidden_layers": 12,
  "num_nodes": 1,
  "num_output_tags": 1,
  "num_return_sequences": 1,
  "num_token_types": 3,
  "num_train_epochs": 3,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_scores": false,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "prefix": null,
  "pretrained_lm_model_path": null,
  "pretrained_model_path": "/Users/alex/.cache/torch/transformers/named-models/roberta-base-expanded-embeddings",
  "problem_type": null,
  "pruned_heads": {},
  "remove_invalid_values": false,
  "repetition_penalty": 1.0,
  "return_dict": true,
  "return_dict_in_generate": false,
  "sentence_contextualizer_model_type": "roberta",
  "sentence_embedding_method": "multiheaded-attention",
  "sep_token_id": null,
  "shuffle_data": true,
  "sinusoidal_embeddings": false,
  "spacy_model_file": "en_core_web_lg",
  "split_type": "random",
  "task_specific_params": null,
  "temperature": 1.0,
  "tie_encoder_decoder": false,
  "tie_word_embeddings": true,
  "tokenizer_class": null,
  "top_k": 50,
  "top_p": 1.0,
  "torch_dtype": "float32",
  "torchscript": false,
  "train_data_file": "/Users/alex/Projects/usc-research/source-exploration/models_neural/quote_attribution/data/our-annotated-data__stage-2.tsv",
  "train_on_none": false,
  "transformer_attention_probs_dropout_prob": 0.1,
  "transformer_hidden_dropout_prob": 0.1,
  "transformers_version": "4.12.2",
  "type_vocab_size": 1,
  "use_bfloat16": false,
  "use_cache": false,
  "use_cpu": false,
  "use_deepspeed": false,
  "use_doc_emb": false,
  "use_headline": false,
  "use_headline_embs": false,
  "use_positional": false,
  "vocab_size": 50265,
  "warmup_steps": 0,
  "weight_decay": 0
}
"""

In [692]:
server_dict = json.loads(t_server)

In [693]:
local_dict = json.loads(t_local)

In [698]:
pd.concat([
    pd.Series(server_dict).to_frame('server'),
    pd.Series(local_dict).to_frame('local')
], axis=1).loc[lambda df: df['server'] != df['local']]

,server,local
bad_words_ids,None,None
classifier_dropout,None,None
cross_attention_hidden_size,None,None
decoder_start_token_id,None,None
downsample_negative_data,1.0,0.1
eval_data_file,None,None
finetuning_task,None,None
forced_bos_token_id,None,None
forced_eos_token_id,None,None
freeze_encoder_layers,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",[]


In [703]:
import sys
sys.path.insert(0, '../models_neural/src')
import coref_resolution_util as c

In [704]:
predictor, nlp = c.load_models()

AttributeError: module 'coref_resolution_util' has no attribute 'load_models'

In [705]:
fuzzy = FuzzyIntersectionStrategy(predictor, nlp)

NameError: name 'FuzzyIntersectionStrategy' is not defined

In [706]:
 print_clusters(doc, intersection_strategy.clusters(text))

NameError: name 'print_clusters' is not defined

In [17]:
import pandas as pd
import glob

assignment_df = pd.read_csv('../app/data/2022-07-08__annotator-assignments.csv.gz', index_col=0)
completed_tasks = glob.glob('../app/data/output_data_affil-role/*/*')
rewritten_completed_tasks = list(map(lambda x: 
                                     x.replace('output_data_affil-role', 'input_data')
                                     .replace('annotated', 'to-annotate')
                                     , completed_tasks))

completed_df = pd.Series(rewritten_completed_tasks).to_frame('fn').assign(done=True)

assignment_df = (
    assignment_df
    .merge(completed_df, how='left', left_on='fn', right_on='fn')
    .assign(done=lambda df: df['done'].fillna(False))
)

In [20]:
assignment_df.loc[lambda df: df['annotator']=='alex']['done'].value_counts()

False    213
True     169
Name: done, dtype: int64

In [21]:
assignment_df

,fn,annotator,file_id,done
0,../app/data/input_data/14/to-annotate-746.json,alex,746.json,False
1,../app/data/input_data/7/to-annotate-364.json,alex,364.json,False
2,../app/data/input_data/16/to-annotate-806.json,alex,806.json,True
3,../app/data/input_data/18/to-annotate-917.json,alex,917.json,True
4,../app/data/input_data/16/to-annotate-814.json,alex,814.json,False
...,...,...,...,...
760,../app/data/input_data/15/to-annotate-762.json,james,762.json,False
761,../app/data/input_data/12/to-annotate-609.json,james,609.json,False
762,../app/data/input_data/2/to-annotate-139.json,james,139.json,False
763,../app/data/input_data/15/to-annotate-775.json,james,775.json,False


In [33]:
alex_portion_assignment_df = (assignment_df
 .loc[lambda df: df['annotator']=='alex']
 .loc[lambda df: df['done'] == False]
 .assign(annotator=np.random.choice(new_annotators, size=213))
)

In [40]:
new_annotator_df = pd.concat([
    alex_portion_assignment_df,
    assignment_df.loc[lambda df: df['annotator'] == 'james']
]).drop('done', axis=1)

In [42]:
new_annotator_df.to_csv('../app/data/2022-08-18__annotator-assignments.csv.gz')

In [47]:
new_annotator_df.loc[lambda df: df['file_id'] == '8.json']

,fn,annotator,file_id
274,../app/data/input_data/0/to-annotate-8.json,alex,8.json


In [28]:
import numpy as np

In [29]:
new_annotators = ['alex', 'hari', 'khushbu']